In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [5]:
%run "{project_path}\llm_custom_apps\common_code\common_func.ipynb"

13


In [6]:
def extract_excel_structure(file_path):
    """
    Extracts information from an Excel file and returns a list of dictionaries.

    Args:
        file_path (str): Path to the Excel file.

    Returns:
        List[dict]: List of dictionaries where the key is the sheet name, and the value is a dictionary 
        with column headers as keys, each having a nested dictionary with 'ColumnID' and 'ColumnValue'.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the file is not a valid Excel file.
        Exception: For other unexpected errors.
    """
    try:
        workbook = openpyxl.load_workbook(file_path, data_only=False)  # Ensure formulas are preserved
        result = []

        for sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]
            sheet_data = {}

            # Iterate through columns
            for col_idx, column in enumerate(sheet.iter_cols(min_row=1, max_row=1), start=1):
                header_cell = column[0]
                # Always get the resolved value for headers (evaluate formula if any, otherwise hardcoded value)
                header = header_cell.value
                if header is not None:
                    col_letter = sheet.cell(row=1, column=col_idx).column_letter

                    # Get cell values for the column
                    cell = sheet.cell(row=2, column=col_idx)
                    if cell.value is not None:
                        if cell.data_type == 'f':  # Check if the cell contains a formula
                            column_value = cell.formula if hasattr(cell, 'formula') else cell.value
                        else:
                            column_value = cell.value

                        sheet_data[header] = {
                            'ColumnID': col_letter,
                            'ColumnValue': column_value
                        }

            result.append({sheet_name: sheet_data})

        return result

    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    except ValueError as ve:
        raise ValueError(f"The file '{file_path}' is not a valid Excel file: {ve}")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")
